# LangChain 고급 프롬프팅 기법

이 노트북에서는 LangChain의 다양한 고급 프롬프팅 기법을 살펴보겠습니다:

- [YAML 파일로 템플릿 만들기](#yaml-파일로-템플릿-만들기): 외부 파일에서 프롬프트 템플릿 로드
- [ChatPromptTemplate](#chatprompttemplate): 대화형 형식의 프롬프트 템플릿 사용
- [MessagePlaceholder](#messageplaceholder): 메시지 목록을 프롬프트에 삽입
- [FewShotPromptTemplate](#fewshotprompttemplate): 예제를 활용한 프롬프트 템플릿
- [Example Selector](#example-selector): 프롬프트에 사용할 예제 선택
- [FewShotChatMessagePromptTemplate](#fewshotchatmessageprompttemplate): 메시지 형식의 few-shot 프롬프트
- [LangChain Hub](#langchain-hub): 프롬프트 공유 및 재사용

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langsmith import Client

client = Client()

## YAML 파일로 템플릿 만들기

YAML 파일에 프롬프트 템플릿을 저장하고 불러와 사용할 수 있습니다. 이 방법은 프롬프트를 코드와 분리하여 관리하기 좋습니다.

In [ ]:
from langchain_teddynote.prompts import load_prompt

prompt = load_prompt("prompts/fruit_color.yaml", encoding="utf-8")
prompt

In [ ]:
prompt.format(fruit="사과")

In [ ]:
prompt2 = load_prompt("prompts/capital.yaml")
print(prompt2.format(country="대한민국"))

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

chain = (
    prompt2 | 
    ChatOpenAI(model_name="gpt-4o", temperature=0) | 
    StrOutputParser()
)
for chunk in chain.stream({"country": "대한민국"}):
    print(chunk, end="")  # 실시간으로 출력

## ChatPromptTemplate

대화형 형식의 템플릿을 사용하고 싶을 때 역할을 부여하고 사용하는 방법입니다.

**role**
- `"system"`: 시스템 설정 메시지입니다. - 페르소나 부여
- `"human"`: 사용자 입력 메시지입니다.
- `"ai"`: AI의 답변 메시지입니다.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("{country}의 수도는 어디인가요?")
chat_prompt

In [ ]:
chat_prompt.format(country="대한민국")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        # role, message
        ("system", "당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name} 입니다."),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}"),
    ]
)

# 챗 message를 생성합니다.
messages = chat_template.format_messages(
    name="테디", user_input="당신의 이름은 무엇입니까?"
)
messages

In [ ]:
llm = ChatOpenAI(model="gpt-4o")

llm.invoke(messages).content

## MessagePlaceholder

`MessagePlaceholder`는 메시지 프롬프트 템플릿에 어떤 역할을 사용해야 할지 확실하지 않거나 서식 지정 중에 메시지 목록을 삽입하려는 경우 유용합니다.

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.",
        ),
        MessagesPlaceholder(variable_name="conversation"),
        ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다."),
    ]
)
chat_prompt

In [ ]:
formatted_chat_prompt = chat_prompt.format(
    word_count=7,
    conversation=[
        ("human", "안녕하세요! 저는 오늘 새로 입사한 홍기 입니다. 만나서 반갑습니다."),
        ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
    ],
)

print(formatted_chat_prompt)

In [ ]:
# chain 생성
chain = chat_prompt | llm | StrOutputParser()

In [ ]:
# chain 실행 및 결과확인
chain.invoke(
    {
        "word_count": 5,
        "conversation": [
            (
                "human",
                "안녕하세요! 저는 오늘 새로 입사한 홍기 입니다. 만나서 반갑습니다.",
            ),
            ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
        ],
    }
)

## FewShotPromptTemplate

Few-shot 프롬프팅은 모델에게 몇 가지 예시를 보여주고 유사한 작업을 수행하도록 하는 기법입니다.

- 하나의 답변 예시로 보여주면 one-shot 프롬프팅
- 두 개 이상의 답변 예시로 보여주면 few-shot 프롬프팅

In [ ]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": "이 질문에 추가 질문이 필요한가요: 예.\n추가 질문: 스티브 잡스는 몇 살에 사망했나요?\n중간 답변: 스티브 잡스는 56세에 사망했습니다.\n추가 질문: 아인슈타인은 몇 살에 사망했나요?\n중간 답변: 아인슈타인은 76세에 사망했습니다.\n최종 답변은: 아인슈타인\n"
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": "이 질문에 추가 질문이 필요한가요: 예.\n추가 질문: 네이버의 창립자는 누구인가요?\n중간 답변: 네이버는 이해진에 의해 창립되었습니다.\n추가 질문: 이해진은 언제 태어났나요?\n중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.\n최종 답변은: 1967년 6월 22일\n"
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": "이 질문에 추가 질문이 필요한가요: 예.\n추가 질문: 율곡 이이의 어머니는 누구인가요?\n중간 답변: 율곡 이이의 어머니는 신사임당입니다.\n추가 질문: 신사임당은 언제 태어났나요?\n중간 답변: 신사임당은 1504년에 태어났습니다.\n추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?\n중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.\n최종 답변은: 연산군\n"
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": "이 질문에 추가 질문이 필요한가요: 예.\n추가 질문: 올드보이의 감독은 누구인가요?\n중간 답변: 올드보이의 감독은 박찬욱입니다.\n추가 질문: 박찬욱은 어느 나라 출신인가요?\n중간 답변: 박찬욱은 대한민국 출신입니다.\n추가 질문: 기생충의 감독은 누구인가요?\n중간 답변: 기생충의 감독은 봉준호입니다.\n추가 질문: 봉준호는 어느 나라 출신인가요?\n중간 답변: 봉준호는 대한민국 출신입니다.\n최종 답변은: 예\n"
    }
]

In [ ]:
example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)

print(example_prompt.format(**examples[0]))

In [ ]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
final_prompt = prompt.format(question=question)
print(final_prompt)

In [ ]:
from langchain_teddynote.messages import stream_response

# 결과 출력
answer = llm.stream(final_prompt)
stream_response(answer)

## Example Selector

프롬프트에 들어갈 예제를 필요에 따라 선택하여 사용하는 기법입니다. 다양한 Example Selector가 있습니다:

- `SemanticSimilarityExampleSelector`: 유사도 계산을 통해 적절한 예제를 선정
- `MaxMarginalRelevanceExampleSelector`: 다양성과 관련성을 모두 고려하여 예시 선택

In [ ]:
from langchain_core.example_selectors import (
    MaxMarginalRelevanceExampleSelector,
    SemanticSimilarityExampleSelector,
)
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# Vector DB 생성 (저장소 이름, 임베딩 클래스)
chroma = Chroma("example_selector", OpenAIEmbeddings())

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 여기에는 선택 가능한 예시 목록이 있습니다.
    examples,
    # 여기에는 의미적 유사성을 측정하는 데 사용되는 임베딩을 생성하는 임베딩 클래스가 있습니다.
    OpenAIEmbeddings(),
    # 여기에는 임베딩을 저장하고 유사성 검색을 수행하는 데 사용되는 VectorStore 클래스가 있습니다.
    Chroma,
    # 이것은 생성할 예시의 수입니다.
    k=1,
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"

# 입력과 가장 유사한 예시를 선택합니다.
selected_examples = example_selector.select_examples({"question": question})

print(f"입력에 가장 유사한 예시:\n{question}\n")
for example in selected_examples:
    print(f'question:\n{example["question"]}')
    print(f'answer:\n{example["answer"]}')

In [ ]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

# 체인 생성
chain = prompt | llm

In [ ]:
# 결과 출력
answer = chain.stream(
    {"question": "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"}
)
stream_response(answer)

## FewShotChatMessagePromptTemplate

Few-shot 프롬프팅을 메시지 형식으로 구현한 것으로, 대화형 컨텍스트에서 예제를 활용할 수 있습니다.

In [ ]:
examples = [
    {
        "instruction": "당신은 회의록 작성 전문가 입니다. 주어진 정보를 바탕으로 회의록을 작성해 주세요",
        "input": "2023년 12월 25일, XYZ 회사의 마케팅 전략 회의가 오후 3시에 시작되었다. 회의에는 마케팅 팀장인 김수진, 디지털 마케팅 담당자인 박지민, 소셜 미디어 관리자인 이준호가 참석했다. 회의의 주요 목적은 2024년 상반기 마케팅 전략을 수립하고, 새로운 소셜 미디어 캠페인에 대한 아이디어를 논의하는 것이었다. 팀장인 김수진은 최근 시장 동향에 대한 간략한 개요를 제공했으며, 이어서 각 팀원이 자신의 분야에서의 전략적 아이디어를 발표했다.",
        "answer": "회의록: XYZ 회사 마케팅 전략 회의\n일시: 2023년 12월 25일\n장소: XYZ 회사 회의실\n참석자: 김수진 (마케팅 팀장), 박지민 (디지털 마케팅 담당자), 이준호 (소셜 미디어 관리자)\n\n1. 개회\n   - 회의는 김수진 팀장의 개회사로 시작됨.\n   - 회의의 목적은 2024년 상반기 마케팅 전략 수립 및 새로운 소셜 미디어 캠페인 아이디어 논의.\n\n2. 시장 동향 개요 (김수진)\n   - 김수진 팀장은 최근 시장 동향에 대한 분석을 제시.\n   - 소비자 행동 변화와 경쟁사 전략에 대한 통찰 공유.\n\n3. 디지털 마케팅 전략 (박지민)\n   - 박지민은 디지털 마케팅 전략에 대해 발표.\n   - 온라인 광고와 SEO 최적화 방안에 중점을 둠.\n\n4. 소셜 미디어 캠페인 (이준호)\n   - 이준호는 새로운 소셜 미디어 캠페인에 대한 아이디어를 제안.\n   - 인플루언서 마케팅과 콘텐츠 전략에 대한 계획을 설명함.\n\n5. 종합 논의\n   - 팀원들 간의 아이디어 공유 및 토론.\n   - 각 전략에 대한 예산 및 자원 배분에 대해 논의.\n\n6. 마무리\n   - 다음 회의 날짜 및 시간 확정.\n   - 회의록 정리 및 배포는 박지민 담당.\n"
    },
    {
        "instruction": "당신은 요약 전문가 입니다. 다음 주어진 정보를 바탕으로 내용을 요약해 주세요",
        "input": "이 문서는 '지속 가능한 도시 개발을 위한 전략'에 대한 20페이지 분량의 보고서입니다. 보고서는 지속 가능한 도시 개발의 중요성, 현재 도시화의 문제점, 그리고 도시 개발을 지속 가능하게 만들기 위한 다양한 전략을 포괄적으로 다루고 있습니다. 이 보고서는 또한 성공적인 지속 가능한 도시 개발 사례를 여러 국가에서 소개하고, 이러한 사례들을 통해 얻은 교훈을 요약하고 있습니다.",
        "answer": "문서 요약: 지속 가능한 도시 개발을 위한 전략 보고서\n\n- 중요성: 지속 가능한 도시 개발이 필수적인 이유와 그에 따른 사회적, 경제적, 환경적 이익을 강조.\n- 현 문제점: 현재의 도시화 과정에서 발생하는 주요 문제점들, 예를 들어 환경 오염, 자원 고갈, 불평등 증가 등을 분석.\n- 전략: 지속 가능한 도시 개발을 달성하기 위한 다양한 전략 제시. 이에는 친환경 건축, 대중교통 개선, 에너지 효율성 증대, 지역사회 참여 강화 등이 포함됨.\n- 사례 연구: 전 세계 여러 도시의 성공적인 지속 가능한 개발 사례를 소개. 예를 들어, 덴마크의 코펜하겐, 일본의 요코하마 등의 사례를 통해 실현 가능한 전략들을 설명.\n- 교훈: 이러한 사례들에서 얻은 주요 교훈을 요약. 강조된 교훈에는 다각적 접근의 중요성, 지역사회와의 협력, 장기적 계획의 필요성 등이 포함됨.\n\n이 보고서는 지속 가능한 도시 개발이 어떻게 현실적이고 효과적인 형태로 이루어질 수 있는지에 대한 심도 있는 분석을 제공합니다."
    },
    {
        "instruction": "당신은 문장 교정 전문가 입니다. 다음 주어진 문장을 교정해 주세요",
        "input": "우리 회사는 새로운 마케팅 전략을 도입하려고 한다. 이를 통해 고객과의 소통이 더 효과적이 될 것이다.",
        "answer": "본 회사는 새로운 마케팅 전략을 도입함으로써, 고객과의 소통을 보다 효과적으로 개선할 수 있을 것으로 기대된다."
    }
]

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.example_selectors import (
    SemanticSimilarityExampleSelector,
)
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

chroma = Chroma("fewshot_chat", OpenAIEmbeddings())

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{instruction}:\n{input}"),
        ("ai", "{answer}"),
    ]
)

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 여기에는 선택 가능한 예시 목록이 있습니다.
    examples,
    # 여기에는 의미적 유사성을 측정하는 데 사용되는 임베딩을 생성하는 임베딩 클래스가 있습니다.
    OpenAIEmbeddings(),
    # 여기에는 임베딩을 저장하고 유사성 검색을 수행하는 데 사용되는 VectorStore 클래스가 있습니다.
    chroma,
    # 이것은 생성할 예시의 수입니다.
    k=1,
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
)

In [ ]:
question = {
    "instruction": "회의록을 작성해 주세요",
    "input": "2023년 12월 26일, ABC 기술 회사의 제품 개발 팀은 새로운 모바일 애플리케이션 프로젝트에 대한 주간 진행 상황 회의를 가졌다. 이 회의에는 프로젝트 매니저인 최현수, 주요 개발자인 황지연, UI/UX 디자이너인 김태영이 참석했다. 회의의 주요 목적은 프로젝트의 현재 진행 상황을 검토하고, 다가오는 마일스톤에 대한 계획을 수립하는 것이었다. 각 팀원은 자신의 작업 영역에 대한 업데이트를 제공했고, 팀은 다음 주까지의 목표를 설정했다.",
}

# 입력과 가장 유사한 예시를 선택합니다.
selected_examples = example_selector.select_examples(question)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        few_shot_prompt,
        ("human", "{instruction}\n{input}"),
    ]
)

# chain 생성
chain = final_prompt | llm

# 실행 및 결과 출력
answer = chain.stream(question)
stream_response(answer)

## LangChain Hub 사용하기

LangChain Hub를 통해 잘 구성된 프롬프트를 공유하고 재사용할 수 있습니다.

In [ ]:
from langchain import hub

# 가장 최신 버전의 프롬프트를 가져옵니다.
prompt = hub.pull("rlm/rag-prompt")

# 프롬프트 내용 출력
print(prompt)

In [ ]:
# 특정 버전의 프롬프트를 가져오려면 버전 해시를 지정하세요
prompt = hub.pull("rlm/rag-prompt:50442af1")
prompt

In [ ]:
# 직접 프롬프트 만들고 허브에 올리기
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "주어진 내용을 바탕으로 다음 문장을 요약하세요. 답변은 반드시 한글로 작성하세요\n\nCONTEXT: {context}\n\nSUMMARY:"
)
prompt

In [ ]:
# 허브에 프롬프트 업로드하기 (실제 계정 필요)
# hub.push("your_username/your_repo_name", prompt)